Importing the necessary libraries.

In [1]:
import csv
import numpy as np
import pandas as pd

Loading the cleaned dataset.

In [2]:
CSV_FILEPATH = 'dataset/dataset_clean.csv'
data = pd.read_csv(CSV_FILEPATH)
data = data.replace({"  ": " "}, regex=True)
data

,Disease,Symptom,Occurence
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2164,pulmonary tuberculosis,nausea,42.0
2165,pulmonary tuberculosis,vomiting,42.0
2166,pulmonary tuberculosis,fatigue,42.0
2167,pulmonary tuberculosis,chill,42.0


### Creating Symptoms ID data frame

Extract unique values from the `Symptom` column.

In [3]:
symptoms = data['Symptom'].unique()
symptoms

array(['pain chest', 'shortness of breath', 'dizziness', 'asthenia',
       'fall', 'syncope', 'vertigo', 'sweat', 'sweating increased',
       'palpitation', 'nausea', 'angina pectoris', 'pressure chest',
       'polyuria', 'polydypsia', 'orthopnea', 'rale', 'unresponsiveness',
       'mental status changes', 'vomiting', 'labored breathing',
       'feeling suicidal', 'suicidal', 'hallucinations auditory',
       'feeling hopeless', 'weepiness', 'sleeplessness',
       'motor retardation', 'irritable mood', 'blackout',
       'mood depressed', 'hallucinations visual', 'worry', 'agitation',
       'tremor', 'intoxication', 'verbal auditory hallucinations',
       'energy increased', 'difficulty', 'nightmare',
       'unable to concentrate', 'homelessness', 'hypokinesia',
       'dyspnea on exertion', 'chest tightness', 'cough', 'fever',
       'decreased translucency', 'productive cough', 'pleuritic pain',
       'yellow sputum', 'breath sounds decreased', 'chill', 'rhonchus',
       '

Create an empty data frame for symptoms.

In [4]:
sym_df = pd.DataFrame(columns=['syd','symptom'])

Iterate through the list of symptoms and add an entry to the symptoms data frame.

In [5]:
id = 1
for symptom in symptoms:
  entry = {'syd': id, 'symptom': symptom}
  sym_df = sym_df.append(entry, ignore_index=True)
  id += 1
sym_df

/tmp/ipykernel_2119/2617087695.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sym_df = sym_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/2617087695.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sym_df = sym_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/2617087695.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sym_df = sym_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/2617087695.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sym_df = sym_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/2617087695.py:4: FutureWarning: The frame.append method is deprecated and will be removed from p

,syd,symptom
0,1,pain chest
1,2,shortness of breath
2,3,dizziness
3,4,asthenia
4,5,fall
...,...,...
400,401,air fluid level
401,402,catching breath
402,403,large-for-dates fetus
403,404,immobile


Export the symptom data frame as a `CSV` file.

In [6]:
CSV_FILEPATH = 'dataset/symptoms_id.csv'
sym_df.to_csv(CSV_FILEPATH, index=False)

### Creating Diseases ID data frame

Extract unique values from the `Symptom` column.

In [7]:
diseases = data['Disease'].unique()
diseases

array(['hypertensive disease', 'diabetes', 'depression mental',
       'depressive disorder', 'coronary arteriosclerosis',
       'coronary heart disease', 'pneumonia', 'failure heart congestive',
       'accident cerebrovascular', 'asthma', 'myocardial infarction',
       'hypercholesterolemia', 'infection', 'infection urinary tract',
       'anemia', 'chronic obstructive airway disease', 'dementia',
       'insufficiency renal', 'confusion', 'degenerative polyarthritis',
       'hypothyroidism', 'anxiety state', 'malignant neoplasms',
       'primary malignant neoplasm',
       'acquired immuno-deficiency syndrome', 'HIV', 'hiv infections',
       'cellulitis', 'gastroesophageal reflux disease', 'septicemia',
       'systemic infection', 'sepsis (invertebrate)',
       'deep vein thrombosis', 'dehydration', 'neoplasm',
       'embolism pulmonary', 'epilepsy', 'cardiomyopathy',
       'chronic kidney failure', 'carcinoma', 'hepatitis C',
       'peripheral vascular disease', 'psychoti

Create an empty data frame for diseases.

In [8]:
dia_df = pd.DataFrame(columns=['did', 'disease'])

Iterate through the list of diseases and add an entry to the diseases data frame.

In [9]:
id = 1
for disease in diseases:
  entry = {'did': id, 'disease': disease}
  dia_df = dia_df.append(entry, ignore_index=True)
  id += 1
dia_df

/tmp/ipykernel_2119/1183902656.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dia_df = dia_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/1183902656.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dia_df = dia_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/1183902656.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dia_df = dia_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/1183902656.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dia_df = dia_df.append(entry, ignore_index=True)
/tmp/ipykernel_2119/1183902656.py:4: FutureWarning: The frame.append method is deprecated and will be removed from p

,did,disease
0,1,hypertensive disease
1,2,diabetes
2,3,depression mental
3,4,depressive disorder
4,5,coronary arteriosclerosis
...,...,...
147,148,decubitus ulcer
148,149,polyneuropathy
149,150,refractive error
150,151,acute nasopharyngitis


Export the diseases data frame as a `CSV` file.

In [10]:
CSV_FILEPATH = 'dataset/diseases_id.csv'
dia_df.to_csv(CSV_FILEPATH, index=False)

### Creating the sparse matrix

Append the symptoms id to the original data frame.

In [11]:
df = pd.merge(data, sym_df, left_on='Symptom', right_on='symptom',how='left').drop(columns=['symptom'], axis=1)
df

,Disease,Symptom,Occurence,syd
0,hypertensive disease,pain chest,3363.0,1
1,hypertensive disease,shortness of breath,3363.0,2
2,hypertensive disease,dizziness,3363.0,3
3,hypertensive disease,asthenia,3363.0,4
4,hypertensive disease,fall,3363.0,5
...,...,...,...,...
2164,pulmonary tuberculosis,nausea,42.0,11
2165,pulmonary tuberculosis,vomiting,42.0,20
2166,pulmonary tuberculosis,fatigue,42.0,102
2167,pulmonary tuberculosis,chill,42.0,53


Create a disease and symptoms data frame.

In [12]:
disease_symptoms_df = pd.DataFrame(columns=['Disease', 'Symptoms'])

Iterate through the list of disease and extract each symptom associated by appending them in a list.

In [13]:
for disease in diseases:
  
  symptoms = []
  for index, row in df.iterrows():
    if disease == row['Disease']:
      symptoms.append(row['Symptom'])

  new_entry = {
    'Disease': disease,
    'Symptoms': symptoms
  }

  disease_symptoms_df = disease_symptoms_df.append(new_entry, ignore_index=True)

disease_symptoms_df

/tmp/ipykernel_2119/2463434039.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  disease_symptoms_df = disease_symptoms_df.append(new_entry, ignore_index=True)
/tmp/ipykernel_2119/2463434039.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  disease_symptoms_df = disease_symptoms_df.append(new_entry, ignore_index=True)
/tmp/ipykernel_2119/2463434039.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  disease_symptoms_df = disease_symptoms_df.append(new_entry, ignore_index=True)
/tmp/ipykernel_2119/2463434039.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  disease_symptoms_df = disease_symptoms_df.append(new_entry, ignore_inde

,Disease,Symptoms
0,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
1,diabetes,"[polyuria, polydypsia, shortness of breath, pa..."
2,depression mental,"[feeling suicidal, suicidal, hallucinations au..."
3,depressive disorder,"[feeling suicidal, suicidal, hallucinations au..."
4,coronary arteriosclerosis,"[pain chest, angina pectoris, shortness of bre..."
...,...,...
147,decubitus ulcer,"[systolic murmur, frail, fever]"
148,polyneuropathy,"[abnormal sensation, numbness, pain, uncoordin..."
149,refractive error,"[vision blurred, fatigue, tired, headache, una..."
150,acute nasopharyngitis,"[sneeze, throat sore, cough, snuffle, stuffy n..."


Create a temporary data frame to isolate the count for each unique disease.

In [14]:
count_df = df[['Disease','Occurence']].drop_duplicates(subset='Disease', keep='first')
count_df

,Disease,Occurence
0,hypertensive disease,3363.0
13,diabetes,1421.0
28,depression mental,1337.0
49,depressive disorder,1337.0
70,coronary arteriosclerosis,1284.0
...,...,...
2138,decubitus ulcer,42.0
2141,polyneuropathy,42.0
2148,refractive error,42.0
2153,acute nasopharyngitis,42.0


Get the sum of all occurence.

In [15]:
total_records = count_df['Occurence'].sum()
print(total_records)

38653.0


Append the diseases id to the original data frame.

In [16]:
df = pd.merge(df, dia_df, left_on='Disease', right_on='disease', how='left').drop(columns=['Disease'], axis=1)
df = pd.merge(df, disease_symptoms_df, left_on='disease', right_on='Disease', how='left').drop(columns=['Disease'], axis=1)
df

,Symptom,Occurence,syd,did,disease,Symptoms
0,pain chest,3363.0,1,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
1,shortness of breath,3363.0,2,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
2,dizziness,3363.0,3,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
3,asthenia,3363.0,4,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
4,fall,3363.0,5,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
...,...,...,...,...,...,...
2164,nausea,42.0,11,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o..."
2165,vomiting,42.0,20,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o..."
2166,fatigue,42.0,102,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o..."
2167,chill,42.0,53,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o..."


Iterate through each entry and calculate the weight of symptom and disease association based solely on symptoms count.

In [17]:
for index, row in df.iterrows():
    symptoms_count = (df['did'] == row['did']).sum()  # Get the total number of symptoms for each disease.
    weight = 1 / symptoms_count # Set the weight of relationship of each symptom to the disease to 1 out of the total number of symptoms a disease have.
    df.loc[index, 'wei'] = weight   # Add a new column for the weight.
df

,Symptom,Occurence,syd,did,disease,Symptoms,wei
0,pain chest,3363.0,1,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.076923
1,shortness of breath,3363.0,2,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.076923
2,dizziness,3363.0,3,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.076923
3,asthenia,3363.0,4,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.076923
4,fall,3363.0,5,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.076923
...,...,...,...,...,...,...,...
2164,nausea,42.0,11,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o...",0.100000
2165,vomiting,42.0,20,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o...",0.100000
2166,fatigue,42.0,102,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o...",0.100000
2167,chill,42.0,53,152,pulmonary tuberculosis,"[cough, barking cough, pain chest, shortness o...",0.100000


Save the sparse matrix as a `CSV` file with the columns set to `syd`, `did`, and `wei`.

In [18]:
CSV_FILEPATH = 'dataset/symptoms-disease_sparse-matrix.csv'
df.to_csv(CSV_FILEPATH, columns=['syd','did','wei'], index=False)

Iterate through each entry again and revise the weight with consideration to the ranking of association.

In [19]:
for index, row in df.iterrows():
    # Get the total number of symptoms for each disease.
    symptoms_count = (df['did'] == row['did']).sum()
    
    total_rank_sum = 0
    for i in range(symptoms_count):
        # Rank sum: With n criteria, rank r receives the weight n-r+1
        rank_sum = symptoms_count-(i+1)+1
        total_rank_sum += rank_sum  # Finding the sum of possible ranking numbers
    
    symptom_rank = (row['Symptoms'].index(row['Symptom']) +1) if (str((row['Symptom'])) != "nan") else 0

    # Rank sum: With n criteria, rank r receives the weight n-r+1
    symptom_rank_sum = symptoms_count-(symptom_rank)+1
    # Set the weight of relationship of each symptom to the disease based on the ranking in association and number of symptoms.
    weight = symptom_rank_sum / total_rank_sum
    df.loc[index, 'wei'] = weight   # Add a new column for the weight.
    
    total_rank_sum = 0 # Reset sum of total rank

Preview the data frame.

In [20]:
df.head(50)

,Symptom,Occurence,syd,did,disease,Symptoms,wei
0,pain chest,3363.0,1,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.142857
1,shortness of breath,3363.0,2,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.131868
2,dizziness,3363.0,3,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.120879
3,asthenia,3363.0,4,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.109890
4,fall,3363.0,5,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.098901
5,syncope,3363.0,6,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.087912
6,vertigo,3363.0,7,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.076923
7,sweat,3363.0,8,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.065934
8,sweating increased,3363.0,9,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.054945
9,palpitation,3363.0,10,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.043956


Save the sparse matrix as another `CSV` file.

In [21]:
CSV_FILEPATH = 'dataset/symptoms-disease-weighted-rank_sparse-matrix.csv'
df.to_csv(CSV_FILEPATH, columns=['syd', 'did', 'wei'], index=False)

Iterate through each entry and apply consideration for disease occurence in the weight of the symptoms.

In [22]:
for index, row in df.iterrows():
    total_records = count_df['Occurence'].sum() # Get the total number of disease occurences.
    disease_weight = (row['Occurence']/total_records) # Get the probability of the disease based on its occurence over the total occurences of all diseases.
    updated_weight = row['wei'] * disease_weight # Multiply the existing probability of symptom to the probability of disease.

    df.loc[index, 'wei'] = updated_weight * 100 # Update the weights. (probability is expressed in percentage, hence the multiplication to 100 to compensate with the number of decimal places for the low value.)

Preview the data frame.

In [23]:
df.head(50)

,Symptom,Occurence,syd,did,disease,Symptoms,wei
0,pain chest,3363.0,1,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",1.242927
1,shortness of breath,3363.0,2,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",1.147317
2,dizziness,3363.0,3,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",1.051707
3,asthenia,3363.0,4,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.956098
4,fall,3363.0,5,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.860488
5,syncope,3363.0,6,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.764878
6,vertigo,3363.0,7,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.669268
7,sweat,3363.0,8,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.573659
8,sweating increased,3363.0,9,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.478049
9,palpitation,3363.0,10,1,hypertensive disease,"[pain chest, shortness of breath, dizziness, a...",0.382439


Save the sparse matrix as another `CSV` file.

In [24]:
CSV_FILEPATH = 'dataset/symptoms-disease-weighted-rank-count_sparse-matrix.csv'
df.to_csv(CSV_FILEPATH, columns=['syd', 'did', 'wei'], index=False)

Create a `CSV` file for disease and symptoms column.

In [25]:
df_mod = df[['disease', 'Symptoms']].drop_duplicates(subset='disease', keep='first')
df_mod.head()

,disease,Symptoms
0,hypertensive disease,"[pain chest, shortness of breath, dizziness, a..."
13,diabetes,"[polyuria, polydypsia, shortness of breath, pa..."
28,depression mental,"[feeling suicidal, suicidal, hallucinations au..."
49,depressive disorder,"[feeling suicidal, suicidal, hallucinations au..."
70,coronary arteriosclerosis,"[pain chest, angina pectoris, shortness of bre..."


In [26]:
CSV_FILEPATH = 'dataset/disease-symptoms-list.csv'
df_mod.to_csv(CSV_FILEPATH, columns=['disease', 'Symptoms'], index=False)